In [34]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score

In [35]:
X_train_in = pd.read_csv("X_train.csv", index_col=0)
y_train_in = pd.read_csv("y_train.csv",index_col=0)
X_val = pd.read_csv("X_test.csv", index_col=0)
st_info = pd.read_csv("studs_info.csv", index_col=0)

In [36]:
X_all = X_train_in.append(X_val, ignore_index=True)

In [37]:
X_train_in.head(1)

,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА
51081,175711,38.03.02 Менеджмент,2018-2019,2.0,4.0,Экзамен,Маркетинг


In [38]:
student_level = []
big_group = []
small_group = []

for i in X_train_in['НАПРАВЛЕНИЕ']:
    student_level.append(i.split('.')[1])
    big_group.append(i.split('.')[0])
    small_group.append(i.split('.')[2][:2])
    
X_train_in['STD_LVL'] = student_level
X_train_in['big_group'] = big_group
X_train_in['small_group'] = small_group

In [39]:
# {'Дифференцированный зачет',5
#  'Зачет',1
#  'Курсовая работа',3
#  'Курсовой проект',4
#  'Экзамен' 2}
X_train_in['АТТЕСТАЦИЯ'].replace(['Дифференцированный зачет','Зачет', 'Курсовая работа', 'Курсовой проект','Экзамен'],[5,1,3,4,2],inplace=True)

In [40]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(X_all.ДИСЦИПЛИНА)

X_train_in.loc[:,"ДИСЦИПЛИНА"] = le.transform(X_train_in.loc[:,"ДИСЦИПЛИНА"])
X_val.loc[:,"ДИСЦИПЛИНА"] = le.transform(X_val.loc[:,"ДИСЦИПЛИНА"])

In [41]:
X_train_in["mark"] = y_train_in["mark"]

In [42]:
X_train_in

,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА,STD_LVL,big_group,small_group,mark
51081,175711,38.03.02 Менеджмент,2018-2019,2.0,4.0,2,408,03,38,02,5
72921,100647,31.05.01 Лечебное дело,2015-2016,3.0,5.0,2,946,05,31,01,5
96438,199311,31.05.03 Стоматология,2019-2020,1.0,1.0,2,1326,05,31,03,5
273044,162002,38.03.02 Менеджмент,2017-2018,2.0,3.0,2,1100,03,38,02,4
371309,108384,31.05.01 Лечебное дело,2018-2019,5.0,9.0,2,713,05,31,01,3
...,...,...,...,...,...,...,...,...,...,...,...
224856,176141,40.03.01 Юриспруденция,2017-2018,1.0,2.0,3,364,03,40,01,3
260998,182794,31.05.01 Лечебное дело,2018-2019,2.0,4.0,2,556,05,31,01,3
57245,162391,31.05.01 Лечебное дело,2016-2017,1.0,2.0,5,228,05,31,01,4
218529,202479,38.03.01 Экономика,2017-2018,1.0,1.0,5,238,03,38,01,4


In [43]:
agg_features = X_train_in.groupby('STD_ID')['mark'].agg(['mean','max','min','std','sum','count', np.median, np.ptp, np.average]).reset_index()

/home/ec2-user/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/ec2-user/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [44]:
new_info = st_info.groupby(st_info.index).first()
new_info = new_info.fillna(0)
new_info.head(1)

,number,Пол,Статус,Дата выпуска,Категория обучения,Форма обучения,Шифр,направление (специальность),,Образование,Дата выдачи,Что именно закончил
STD_ID,,,,,,,,,,,,
57258,67.0,М,СТ,0,ДОГ,З,15.03.05,Конструкторско-технологическое обеспечение маш...,10.0,СР,2002-07-31 00:00:00,"ГИМНАЗИЯ N 2, НОВГОРОД"


In [45]:
X_train_in = pd.merge(X_train_in,agg_features,on='STD_ID')

In [46]:
X_train_in.head(3)

,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА,STD_LVL,big_group,small_group,mark,mean,max,min,std,sum,count,median,ptp,average
0,175711,38.03.02 Менеджмент,2018-2019,2.0,4.0,2,408,03,38,02,5,4.181818,5,3,0.58849,92,22,4.0,2,4.181818
1,175711,38.03.02 Менеджмент,2017-2018,1.0,1.0,2,731,03,38,02,4,4.181818,5,3,0.58849,92,22,4.0,2,4.181818
2,175711,38.03.02 Менеджмент,2017-2018,1.0,2.0,5,254,03,38,02,4,4.181818,5,3,0.58849,92,22,4.0,2,4.181818


In [47]:
# stud_mean_mark = X_train_in.groupby("STD_ID").mean()["mark"]
# X_train_in.loc[:,"stud_mean"] = X_train_in.loc[:,"STD_ID"].map(stud_mean_mark)

In [48]:
X_train_in.shape

(105058, 20)

In [49]:
X_val['АТТЕСТАЦИЯ'].replace(['Дифференцированный зачет','Зачет', 'Курсовая работа', 'Курсовой проект','Экзамен'],[5,1,3,4,2],inplace=True)

In [50]:
X_val.head(3)

,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА
56065,163261,35.03.04 Агрономия,2018-2019,3.0,5.0,5,260
304637,162990,23.03.03 Эксплуатация транспортно-технологичес...,2019-2020,4.0,7.0,2,1028
177437,192595,40.03.01 Юриспруденция,2018-2019,1.0,2.0,2,269


In [51]:
X_val = pd.merge(X_val, agg_features, on='STD_ID', how='left')

In [52]:
student_level = []
big_group = []
small_group = []

for i in X_val['НАПРАВЛЕНИЕ']:
    student_level.append(i.split('.')[1])
    big_group.append(i.split('.')[0])
    small_group.append(i.split('.')[2][:2])
    
X_val['STD_LVL'] = student_level
X_val['big_group'] = big_group
X_val['small_group'] = small_group

In [53]:
X_val.head(3)

,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА,mean,max,min,std,sum,count,median,ptp,average,STD_LVL,big_group,small_group
0,163261,35.03.04 Агрономия,2018-2019,3.0,5.0,5,260,4.095238,5.0,3.0,0.538958,86.0,21.0,4.0,2.0,4.095238,03,35,04
1,162990,23.03.03 Эксплуатация транспортно-технологичес...,2019-2020,4.0,7.0,2,1028,3.896552,5.0,3.0,0.772049,113.0,29.0,4.0,2.0,3.896552,03,23,03
2,192595,40.03.01 Юриспруденция,2018-2019,1.0,2.0,2,269,3.909091,5.0,3.0,0.943880,43.0,11.0,4.0,2.0,3.909091,03,40,01


In [54]:
from sklearn.model_selection import train_test_split

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X_train_in,X_train_in["mark"], shuffle=True, test_size=0.3)

In [56]:
y_test.shape, y_train.shape

((31518,), (73540,))

In [57]:
from catboost import CatBoostRegressor, CatBoostClassifier

In [58]:
train_col = ["КУРС","СЕМЕСТР", "АТТЕСТАЦИЯ", "ДИСЦИПЛИНА", "mean", "std",
             "count", "max", "min", "sum", "STD_LVL", 'median',
             'ptp', 'average', 'big_group', 'small_group']
# X_train["STD_ID"]= X_train["STD_ID"].astype(int)
# X_test["STD_ID"]= X_test["STD_ID"].astype(int)


In [59]:
model = CatBoostClassifier(iterations=1000,  classes_count=6, depth = 7, learning_rate=0.3,)

In [60]:
model.fit(X_train[train_col],y_train)

Found only 3 unique classes in the data, but have defined 6 classes. Probably something is wrong with data.


0:	learn: 0.9603584	total: 11ms	remaining: 11s
1:	learn: 0.8853742	total: 21.4ms	remaining: 10.7s
2:	learn: 0.8410236	total: 31.3ms	remaining: 10.4s
3:	learn: 0.8110065	total: 41.2ms	remaining: 10.3s
4:	learn: 0.7858256	total: 51.1ms	remaining: 10.2s
5:	learn: 0.7711634	total: 61.2ms	remaining: 10.1s
6:	learn: 0.7584768	total: 71.2ms	remaining: 10.1s
7:	learn: 0.7495090	total: 80.8ms	remaining: 10s
8:	learn: 0.7427473	total: 90.7ms	remaining: 9.99s
9:	learn: 0.7376633	total: 100ms	remaining: 9.94s
10:	learn: 0.7340788	total: 110ms	remaining: 9.92s
11:	learn: 0.7295149	total: 120ms	remaining: 9.89s
12:	learn: 0.7267922	total: 130ms	remaining: 9.88s
13:	learn: 0.7248560	total: 140ms	remaining: 9.86s
14:	learn: 0.7221305	total: 150ms	remaining: 9.85s
15:	learn: 0.7186901	total: 160ms	remaining: 9.86s
16:	learn: 0.7172651	total: 171ms	remaining: 9.9s
17:	learn: 0.7156314	total: 183ms	remaining: 10s
18:	learn: 0.7140449	total: 196ms	remaining: 10.1s
19:	learn: 0.7127710	total: 206ms	remaini

161:	learn: 0.6274399	total: 1.87s	remaining: 9.7s
162:	learn: 0.6269828	total: 1.89s	remaining: 9.7s
163:	learn: 0.6264992	total: 1.9s	remaining: 9.69s
164:	learn: 0.6261958	total: 1.91s	remaining: 9.68s
165:	learn: 0.6255027	total: 1.93s	remaining: 9.68s
166:	learn: 0.6253465	total: 1.94s	remaining: 9.67s
167:	learn: 0.6250207	total: 1.95s	remaining: 9.66s
168:	learn: 0.6245858	total: 1.96s	remaining: 9.65s
169:	learn: 0.6244197	total: 1.97s	remaining: 9.64s
170:	learn: 0.6243134	total: 1.99s	remaining: 9.63s
171:	learn: 0.6239516	total: 2s	remaining: 9.62s
172:	learn: 0.6236546	total: 2.01s	remaining: 9.62s
173:	learn: 0.6233756	total: 2.02s	remaining: 9.61s
174:	learn: 0.6231690	total: 2.04s	remaining: 9.6s
175:	learn: 0.6228083	total: 2.05s	remaining: 9.59s
176:	learn: 0.6225493	total: 2.06s	remaining: 9.58s
177:	learn: 0.6221739	total: 2.07s	remaining: 9.58s
178:	learn: 0.6217461	total: 2.09s	remaining: 9.59s
179:	learn: 0.6216351	total: 2.1s	remaining: 9.59s
180:	learn: 0.621183

326:	learn: 0.5826327	total: 4.14s	remaining: 8.52s
327:	learn: 0.5822896	total: 4.15s	remaining: 8.51s
328:	learn: 0.5820562	total: 4.17s	remaining: 8.5s
329:	learn: 0.5818953	total: 4.18s	remaining: 8.48s
330:	learn: 0.5817003	total: 4.19s	remaining: 8.47s
331:	learn: 0.5816246	total: 4.2s	remaining: 8.46s
332:	learn: 0.5813769	total: 4.22s	remaining: 8.44s
333:	learn: 0.5810245	total: 4.23s	remaining: 8.43s
334:	learn: 0.5808128	total: 4.24s	remaining: 8.42s
335:	learn: 0.5806043	total: 4.25s	remaining: 8.41s
336:	learn: 0.5804363	total: 4.27s	remaining: 8.39s
337:	learn: 0.5800450	total: 4.28s	remaining: 8.38s
338:	learn: 0.5797244	total: 4.29s	remaining: 8.37s
339:	learn: 0.5795799	total: 4.3s	remaining: 8.36s
340:	learn: 0.5793058	total: 4.32s	remaining: 8.34s
341:	learn: 0.5791529	total: 4.33s	remaining: 8.33s
342:	learn: 0.5788166	total: 4.34s	remaining: 8.32s
343:	learn: 0.5785986	total: 4.36s	remaining: 8.31s
344:	learn: 0.5782989	total: 4.37s	remaining: 8.29s
345:	learn: 0.5

485:	learn: 0.5508768	total: 6.38s	remaining: 6.75s
486:	learn: 0.5507169	total: 6.4s	remaining: 6.74s
487:	learn: 0.5505268	total: 6.41s	remaining: 6.72s
488:	learn: 0.5504033	total: 6.42s	remaining: 6.71s
489:	learn: 0.5503194	total: 6.44s	remaining: 6.7s
490:	learn: 0.5502556	total: 6.45s	remaining: 6.69s
491:	learn: 0.5500119	total: 6.47s	remaining: 6.68s
492:	learn: 0.5498629	total: 6.48s	remaining: 6.67s
493:	learn: 0.5497414	total: 6.5s	remaining: 6.66s
494:	learn: 0.5496096	total: 6.51s	remaining: 6.64s
495:	learn: 0.5494658	total: 6.53s	remaining: 6.63s
496:	learn: 0.5493352	total: 6.54s	remaining: 6.62s
497:	learn: 0.5491431	total: 6.55s	remaining: 6.61s
498:	learn: 0.5490303	total: 6.57s	remaining: 6.6s
499:	learn: 0.5488518	total: 6.59s	remaining: 6.59s
500:	learn: 0.5487048	total: 6.6s	remaining: 6.57s
501:	learn: 0.5485188	total: 6.62s	remaining: 6.56s
502:	learn: 0.5483572	total: 6.63s	remaining: 6.55s
503:	learn: 0.5482173	total: 6.65s	remaining: 6.54s
504:	learn: 0.548

655:	learn: 0.5239679	total: 8.85s	remaining: 4.64s
656:	learn: 0.5238028	total: 8.87s	remaining: 4.63s
657:	learn: 0.5237066	total: 8.89s	remaining: 4.62s
658:	learn: 0.5235544	total: 8.9s	remaining: 4.61s
659:	learn: 0.5234165	total: 8.92s	remaining: 4.59s
660:	learn: 0.5232343	total: 8.93s	remaining: 4.58s
661:	learn: 0.5230801	total: 8.95s	remaining: 4.57s
662:	learn: 0.5229929	total: 8.96s	remaining: 4.55s
663:	learn: 0.5228075	total: 8.97s	remaining: 4.54s
664:	learn: 0.5226566	total: 8.99s	remaining: 4.53s
665:	learn: 0.5225035	total: 9.01s	remaining: 4.52s
666:	learn: 0.5223741	total: 9.02s	remaining: 4.5s
667:	learn: 0.5221693	total: 9.04s	remaining: 4.49s
668:	learn: 0.5219806	total: 9.05s	remaining: 4.48s
669:	learn: 0.5218067	total: 9.07s	remaining: 4.46s
670:	learn: 0.5216806	total: 9.08s	remaining: 4.45s
671:	learn: 0.5214758	total: 9.1s	remaining: 4.44s
672:	learn: 0.5213160	total: 9.11s	remaining: 4.43s
673:	learn: 0.5211154	total: 9.13s	remaining: 4.41s
674:	learn: 0.5

818:	learn: 0.5019471	total: 11.3s	remaining: 2.5s
819:	learn: 0.5017303	total: 11.3s	remaining: 2.49s
820:	learn: 0.5016132	total: 11.4s	remaining: 2.48s
821:	learn: 0.5014440	total: 11.4s	remaining: 2.46s
822:	learn: 0.5013840	total: 11.4s	remaining: 2.45s
823:	learn: 0.5012377	total: 11.4s	remaining: 2.43s
824:	learn: 0.5010766	total: 11.4s	remaining: 2.42s
825:	learn: 0.5008759	total: 11.4s	remaining: 2.4s
826:	learn: 0.5007336	total: 11.4s	remaining: 2.39s
827:	learn: 0.5005880	total: 11.4s	remaining: 2.38s
828:	learn: 0.5003781	total: 11.5s	remaining: 2.36s
829:	learn: 0.5002841	total: 11.5s	remaining: 2.35s
830:	learn: 0.5002055	total: 11.5s	remaining: 2.33s
831:	learn: 0.5000926	total: 11.5s	remaining: 2.32s
832:	learn: 0.4999197	total: 11.5s	remaining: 2.31s
833:	learn: 0.4997872	total: 11.5s	remaining: 2.29s
834:	learn: 0.4997553	total: 11.5s	remaining: 2.28s
835:	learn: 0.4996176	total: 11.5s	remaining: 2.27s
836:	learn: 0.4994639	total: 11.6s	remaining: 2.25s
837:	learn: 0.

980:	learn: 0.4826680	total: 13.4s	remaining: 259ms
981:	learn: 0.4825648	total: 13.4s	remaining: 246ms
982:	learn: 0.4824851	total: 13.4s	remaining: 232ms
983:	learn: 0.4823862	total: 13.4s	remaining: 218ms
984:	learn: 0.4822575	total: 13.4s	remaining: 205ms
985:	learn: 0.4821419	total: 13.4s	remaining: 191ms
986:	learn: 0.4820104	total: 13.5s	remaining: 177ms
987:	learn: 0.4818993	total: 13.5s	remaining: 164ms
988:	learn: 0.4816552	total: 13.5s	remaining: 150ms
989:	learn: 0.4815822	total: 13.5s	remaining: 136ms
990:	learn: 0.4815093	total: 13.5s	remaining: 123ms
991:	learn: 0.4813783	total: 13.5s	remaining: 109ms
992:	learn: 0.4812696	total: 13.5s	remaining: 95.4ms
993:	learn: 0.4811822	total: 13.5s	remaining: 81.8ms
994:	learn: 0.4810752	total: 13.6s	remaining: 68.2ms
995:	learn: 0.4810043	total: 13.6s	remaining: 54.5ms
996:	learn: 0.4809465	total: 13.6s	remaining: 40.9ms
997:	learn: 0.4807745	total: 13.6s	remaining: 27.3ms
998:	learn: 0.4805873	total: 13.6s	remaining: 13.6ms
999:	

In [28]:
mean_absolute_error(y_test, model.predict(X_test[train_col])), r2_score(y_test, model.predict(X_test[train_col]))

(0.33279395900755127, 0.34944776177528314)

In [29]:
y_pred = pd.read_csv("sample_submission.csv",index_col=0)
y_pred["mark"] = model.predict(X_val[train_col])

In [30]:
y_pred[y_pred["mark"]<3]

,mark
index,


In [31]:
#y_pred[y_pred["mark"]<3] = 3

In [32]:
#print(model.predict(X_val[train_col]).shape, pd.read_csv("sample_submission.csv",index_col=0).shape)

In [33]:
import time
import os

current_timestamp = int(time.time())
submission_path = 'Submissions/{}.csv'.format(current_timestamp)

if not os.path.exists('Submissions'):
    os.makedirs('Submissions')

print(submission_path)
y_pred.to_csv(submission_path, index=True)

Submissions/1585235811.csv
